<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Timeliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [32]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [33]:
if os.path.exists('PGM-Project'):
  !git -C PGM-Project/ pull
else:
  u = input('GitHub username: ')
  p = getpass('GitHub password: ')
  !git clone https://$u:$p@github.com/remidion/PGM-Project.git
  del u, p

Already up to date.


# Load Data

In [34]:
out_df = pd.read_csv("PGM-Project/data/route_01_full.csv", index_col=[0,1], header=[0,1])
out_df

scheduled           ... timeliness         
stop_sequence                          1.0      2.0  ...        8.0      9.0
service_date route_start_time                        ...                    
2018-08-02   1900-01-01 05:10:00   18600.0  18780.0  ...       Late     Late
             1900-01-01 05:30:00   19800.0  19980.0  ...       Late     Late
             1900-01-01 05:50:00   21000.0  21180.0  ...       Late     Late
             1900-01-01 06:06:00   21960.0  22140.0  ...       Late     Late
             1900-01-01 06:21:00   22860.0  23040.0  ...    On Time  On Time
...                                    ...      ...  ...        ...      ...
2019-09-30   1900-01-01 23:40:00   85200.0  85380.0  ...    On Time  On Time
             1900-01-01 23:54:00   86040.0  86220.0  ...    On Time  On Time
             1900-01-02 00:22:00   87720.0  87900.0  ...      Early    Early
             1900-01-02 00:37:00   88620.0  88800.0  ...    On Time    Early
             1900-01-02 00:52:00   89520.0  89700.0  ...    On Time    Early

[37203 rows x 18 columns]

In [35]:
out2_df = pd.read_csv("PGM-Project/data/route_01_binned.csv", index_col=[0], header=[0,1])
out2_df

service_date scheduled  ... timeliness            route_bin
stop_sequence Unnamed: 1_level_1       1.0  ...        9.0  Unnamed: 20_level_1
0                     2018-08-03   21960.0  ...    On Time  1900-01-01 06:00:00
1                     2018-08-03   23700.0  ...       Late  1900-01-01 06:30:00
2                     2018-08-03   25560.0  ...       Late  1900-01-01 07:00:00
3                     2018-08-03   27300.0  ...      Early  1900-01-01 07:30:00
4                     2018-08-03   28980.0  ...       Late  1900-01-01 08:00:00
...                          ...       ...  ...        ...                  ...
9153                  2019-09-30   81240.0  ...    On Time  1900-01-01 22:30:00
9154                  2019-09-30   83580.0  ...      Early  1900-01-01 23:00:00
9155                  2019-09-30   85200.0  ...    On Time  1900-01-01 23:30:00
9156                  2019-09-30   87720.0  ...      Early  1900-01-02 00:00:00
9157                  2019-09-30   88620.0  ...      Early  1900-01-02 00:30:00

[9158 rows x 20 columns]

In [36]:
n_bins = int(out2_df["route_bin"].nunique())

# Training

In [37]:
X = out_df['scheduled'].to_numpy()
y = out_df['timeliness'].to_numpy()

X2 = out2_df['scheduled'].to_numpy()
y2 = out2_df['timeliness'].to_numpy()

In [38]:
X

array([[18600., 18780., 18960., ..., 19680., 19920., 19980.],
       [19800., 19980., 20160., ..., 20880., 21120., 21180.],
       [21000., 21180., 21360., ..., 22080., 22320., 22380.],
       ...,
       [87720., 87900., 88080., ..., 89040., 89280., 89520.],
       [88620., 88800., 88980., ..., 89940., 90180., 90420.],
       [89520., 89700., 89880., ..., 90840., 91080., 91320.]])

In [39]:
y

array([['On Time', 'On Time', 'On Time', ..., 'Late', 'Late', 'Late'],
       ['On Time', 'On Time', 'On Time', ..., 'Late', 'Late', 'Late'],
       ['On Time', 'On Time', 'On Time', ..., 'Late', 'Late', 'Late'],
       ...,
       ['Late', 'Late', 'Late', ..., 'On Time', 'Early', 'Early'],
       ['On Time', 'On Time', 'On Time', ..., 'On Time', 'On Time',
        'Early'],
       ['On Time', 'On Time', 'On Time', ..., 'Early', 'On Time',
        'Early']], dtype=object)

## Aligned Bins

In [40]:
# Split dataset into training set and test set
test_fraction = 0.2
split_idx = int(X.shape[0] * test_fraction)
X_train, X_test, y_train, y_test = X[:-split_idx], X[-split_idx:], y[:-split_idx], y[-split_idx:]

In [41]:
# Reshape into (Days x time bins x stops)
shape = (X2.shape[0]//n_bins, n_bins, X2.shape[1])

X2 = X2.reshape(shape) 
y2 = y2.reshape(shape) 

In [42]:
# Split dataset #2 into training set and test set
test_fraction = 0.2
split_idx = int(X2.shape[0] * test_fraction)
X2_train, X2_test, y2_train, y2_test = X2[:-split_idx], X2[-split_idx:], y2[:-split_idx], y2[-split_idx:]

In [43]:
X2_train.shape

(193, 38, 9)

In [44]:
X2_test.shape

(48, 38, 9)

## Mode Baseline

In [45]:
from collections import Counter

def baseline(df, verbose=False):
  c = Counter(df.flatten())
  counts = np.array(list(c.values()))
  if verbose: print('Values: ', dict(c.items()))
  ratio = counts/sum(counts)
  mode_performance = max(ratio)
  if verbose: print('Distribution: ', ratio)
  return mode_performance

def b2(df, verbose=True):
  c = Counter(df.flatten())
  myDict = dict(c.items())
  if verbose: print(myDict)
  s = sum(c.values())
  key = max(myDict, key=myDict.get)
  baseline = f'{myDict[key]/s} for "{key}"'
  #print(key, myDict[key]/s)
  return baseline

#print(' y:', b2(y))
#print()
#print('y2:', b2(y2))
#print()

print('y2:', b2(y2_train))

{'Early': 11368, 'On Time': 31270, 'Late': 23368}
y2: 0.4737448110777808 for "On Time"


In [46]:
print(X_train.shape,X2_train.shape, X2_train.reshape(-1,9).shape)
print(y_train.shape,y2_train.shape, y2_train.reshape(-1,9).shape)

(29763, 9) (193, 38, 9) (7334, 9)
(29763, 9) (193, 38, 9) (7334, 9)


# Inference

## 1-d Markov Chain



In [47]:
import math

def get_dtw(knots, x, homogenous=False):
  if homogeneous: return 0
  for k in range(len(knots) - 1):
        if x >= knots[k] and x < knots[k + 1]:
            return k


def parametrization(state_dict, X_tr, y_tr, knots = None, homogeneous = True):
      
    N, T = X_tr.shape

    # Initialize the Transition Matrix.ces
    K = 1 if homogeneous else len(knots)-1
    transition_matrix = np.zeros((K,3,3))
    pi = np.zeros((K, 3))

    for i in range(N):

      # Ground Truth for first stop of sequence
      actual_state = state_dict[y_tr[i,0]] 

      dtw = get_dtw(knots, X_tr[i,0], homogeneous)
      pi[dtw, actual_state] += 1

      for j in range(T - 1):
        actual_state = state_dict[y_tr[i,j]]
        next_state = state_dict[y_tr[i,j + 1]]
        
        dtw = get_dtw(knots, X_tr[i,j], homogeneous)
        transition_matrix[dtw, actual_state, next_state] +=  1

    for k in range(K):

      if sum(pi[k,:]) == 0:
        pi[k,:] = [0,1,0]  # Baseline is On Time
        #pi[k,:] = [1/3, 1/3, 1/3]
      else:
        pi[k,:] = pi[k,:] / sum(pi[k,:])
      
      #Check first divison by 0
      #if transition_matrix[dtw,:,:].sum(axis = 1)[0] == 0 and transition_matrix[dtw,:,:].sum(axis = 1)[1] == 0 and transition_matrix[dtw,:,:].sum(axis = 1)[2] == 0:
      #    transition_matrix[dtw,:,:] =  transition_matrix[dtw - 1,:,:]
      #else:
      if transition_matrix[k,:,:].sum(axis = 1)[0] == 0:
        transition_matrix[k,0,:] = pi[k,:] #[1/3, 1/3, 1/3]
      if transition_matrix[k,:,:].sum(axis = 1)[1] == 0:
        transition_matrix[k,1,:] = pi[k,:] #[1/3, 1/3, 1/3]
      if transition_matrix[k,:,:].sum(axis = 1)[2] == 0:
        transition_matrix[k,2,:] = pi[k,:] #[1/3, 1/3, 1/3]
          
      transition_matrix[k,:,:] = (transition_matrix[k,:,:].T/transition_matrix[k,:,:].sum(axis = 1)).T

    return pi, transition_matrix


def pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots = None, homogeneous = True):

    future_states = []

    dtw = get_dtw(knots, X_tst[0], homogeneous)
    first_stop_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :])

    future_states.append(first_stop_state)

    for i in range(len(X_tst)):
      current_state = state_dict[future_states[-1]]

      dtw = get_dtw(knots, X_tst[i], homogeneous)
      next_state = np.random.choice(list(state_dict.keys()), p = transition_matrix[dtw, current_state, :])
      future_states.append(next_state)

    return future_states


def pred_next_stop_delay(current_state, state_dict, transition_matrix, X_tst, knots, homogeneous = True):

    dtw = get_dtw(knots, X_tst, homogeneous)
    return np.random.choice(list(state_dict.keys()), p = transition_matrix[dtw, state_dict[current_state], :])



In [48]:
#----- Parameters ----- 
slidewindow_len = 60 #Number of previous days to train on
homogeneous = True
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogeneous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X_train.shape
for h in [True, False]:
  homogeneous=h
  print()
  print('homogenous =',h)

  for k in range(slidewindow_len, N):

      X_tr = X_train[k-slidewindow_len:k,:]
      y_tr = y_train[k-slidewindow_len:k,:]
      X_tst = X_train[k,:]
      y_tst = y_train[k,:]
      pi, transition_matrix = parametrization(state_dict, X_tr, y_tr, knots, homogeneous)


      #Predict the next trip sequence of states
      future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, homogeneous)

      for t in range(T):
        if y_tst[t] == future_states[t]:
            accu1 += 1
      

      #Predict the state of the next stop
      dtw = get_dtw(knots, X_tst[0], homogeneous)
      first_state = np.random.choice(list(state_dict.keys()), p = pi[dtw, :]) #Prediction of the first stop state is random
      
      if y_tst[0] == first_state:
          accu2 += 1

      for t in range(1, T):
          next_state = pred_next_stop_delay(y_tst[t-1], state_dict, transition_matrix, X_tst[t], knots, homogeneous)
          
          if y_tst[t] == next_state:
              accu2 += 1


  accu1 = accu1 /((N - slidewindow_len) * T)
  accu2 = accu2 /((N - slidewindow_len) * T)
  print('Sequence:', accu1)
  print('Next', accu2)



homogenous = True


KeyboardInterrupt: ignored

accu1 = Estimate whole sequence \\
accu2 = Estimate state for next stop ID \\
 \\
homogeneous:

*   accu1: 0.43205470637966764
*   accu2: 0.6853260910226772

inhomogeneous (DTW = 1 hour):

*   accu1 0.4616673774306257
*   accu2 0.6461181065248132

inhomogeneous (DTW = peak vs off-peak):

*   accu1 0.44364015891185776
*   accu2 0.6807622382331157



## 2-d Markov chain


In [18]:
# Split dataset #2 into training set and validation set
valid_fraction = 0.2
split_idx = int(X2_train.shape[0] * valid_fraction)
print(X2_train.shape, y2_train.shape)
print('split index', X2_train.shape[0] - split_idx)
X2_tr, X2_val, y2_tr, y2_val = X2_train[:-split_idx], X2_train[-split_idx:], y2_train[:-split_idx], y2_train[-split_idx:]
print(X2_tr.shape, X2_val.shape, y2_tr.shape, y2_val.shape)

(193, 38, 9) (193, 38, 9)
split index 155
(155, 38, 9) (38, 38, 9) (155, 38, 9) (38, 38, 9)


In [60]:
def parametrization_2d(state_dict, X_tr, y_tr, knots=None, stop_homogeneous=True, time_homogeneous=True):
      
  N, T = X_tr.shape

  # Initialize the Transition Matrix.ces
  K = 1 if time_homogeneous else len(knots)-1
  L1 = 1 if stop_homogeneous else T
  L2 = 1 if stop_homogeneous else T-1
  
  P = np.zeros((L2,K,3,3))
  Q = np.zeros((L1,K,3,3))
  pi = np.zeros((K, 3))

  prev_seq = np.zeros(T).astype(int)
  current_seq = np.zeros(T).astype(int)

  for i in range(N):

    # Ground Truth for first stop of sequence
    actual_state = state_dict[y_tr[i,0]] 

    dtw = get_dtw(knots, X_tr[i,0], time_homogeneous)
    pi[dtw, actual_state] += 1

    prev_seq = y_tr[i,:]

    for j in range(T - 1):
      # Update P
      actual_state = state_dict[y_tr[i,j]]
      next_state = state_dict[y_tr[i,j + 1]]
      dtw = get_dtw(knots, X_tr[i,j], time_homogeneous)
      l = 0 if stop_homogeneous else j
      P[l, dtw, actual_state, next_state] +=  1

      #Update Q
      if i > N:  
          next_state = state_dict[y_tr[i + 1,j]]
          dtw = get_dtw(knots, X_tr[i-1,j], time_homogeneous)  # X_tr[i-1,j] since this is the transition from i-1 -> i
          l = 0 if stop_homogeneous else j
          Q[l, dtw, prev_seq[j], current_seq[j]] +=  1
    
  for k in range(K):

    if sum(pi[k,:]) == 0:
      pi[k,:] = [0,1,0]  # Baseline is On Time
      #pi[k,:] = [1/3, 1/3, 1/3]
    else:
      pi[k,:] = pi[k,:] / sum(pi[k,:])
  
    for j in range(L1):
      for r in range(3):
        if Q[j,k,r,:].sum() == 0: Q[j,k,r,:] = pi[k,:] #[1/3, 1/3, 1/3]            
      Q[j,k,:,:] = (Q[j,k,:,:].T/Q[j,k,:,:].sum(axis = 1)).T
      if j < L2:
        for r in range(3):
          if P[j,k,r,:].sum() == 0: P[j,k,r,:] = pi[k,:] #[1/3, 1/3, 1/3]
        P[j,k,:,:] = (P[j,k,:,:].T/P[j,k,:,:].sum(axis = 1)).T

  return pi, P, Q

def pred_stop(previous_sequence, state_dict, pi, P, Q, time_stamp, knots = None, stop_homogeneous = True, time_homogeneous = True, alpha=1, current_sequence=None):
  #x(t+1, h+1) = alpha * x(t, h+1) * P + (1 - alpha) * x(t+1, h) * Q

  T = previous_sequence.shape[0]
  next_state = []

  for t in range(T):
    l = 0 if stop_homogeneous else t
    dtw = get_dtw(knots, time_stamp[t], time_homogeneous)
    vertical = Q[l, dtw, state_dict[previous_sequence[t]], :]
    
    if t==0:
      horizontal = pi[dtw, :] 
    else:
      if current_sequence is None:
        current_state = state_dict[next_state[t-1]]
      else:
        current_state = state_dict[current_sequence[t-1]]
      l = 0 if stop_homogeneous else t-1
      horizontal = P[l, dtw, current_state, :]

    idx = np.argmax(alpha * horizontal + (1-alpha) * vertical)
    next_state.append(list(state_dict.keys())[idx])

  return next_state


In [61]:
def experiment(data, hyperparams):

  train_trips = data[0]  # Unordered list of bus TRIPS - May be empty
  train_timeliness = data[1]  # Associated state labels ^ - May be empty

  print('last continuous-time timestamp:', np.max(train_trips)/3600)

  #----- Parameters ----- 
  print('==== Hyperparameters ====')
  homogeneous = hyperparams[0]
  print('Homogenous =',homogeneous)
  ## homogeneous = 0 : horizontal-only homogenous
  ## homogeneous = 1 : vertical-only homogenous
  ## homogeneous = 2 : 2D homogeneous
  ## homogeneous = 3 : 2D inhomogeneous
  stop_homogeneous = (homogeneous % 2 == 0)  # Case 0 & 2
  time_homogeneous = (homogeneous % 3 != 0)  # Case 1 & 2
  print('Stop Homogenous =',stop_homogeneous)
  print('Time Homogenous =',time_homogeneous)

  ### DTWs time limits (for inhomogeneous chains) ###
  #[0...3600*H] where H = 26  => There are 26 "workable" hours for a bus trip
  inhomogeneous_method = hyperparams[1]
  print('Inhomogeneous Method =', inhomogeneous_method)
  if inhomogeneous_method==0:
    #Peaks period
    knots = [0, 21600, 32400, 50400, 64800, 93600]
  elif inhomogeneous_method==1:
    # Hourly
    knots = [3600*i for i in range(0, 26 + 1)] 

  direction = hyperparams[2]
  ## direction = 0 : horizontal
  ## direction = 1 : vertical
  ## direction = 2 : 2D
  print('Conditional Dependance Direction =', direction)

  ## 2D combination factor x(t+1, h+1) = alpha * x(t, h+1) * P + (1 - alpha) * x(t+1, h) * Q
  ## where 
  ##   t is the stop ID
  ##   h is the hour of the day
  ##   P is the horizontal transition matrix (stop_id-wise)
  ##   Q is the vertical transition matrix (time-wise)
  if direction == 0:
    alpha = 1
  elif direction == 1:
    alpha = 0
  elif direction == 2:
    alpha = hyperparams[3]  # Any value between 0 and 1
    if alpha < 0 or alpha > 1:
      raise Exception('alpha must be between 0 and 1')
  print('alpha =', alpha)

  training_method = hyperparams[4]
  print('Training Method =', training_method)
  ## training_method = 0 : sliding window
  ## training_method = 1 : pre-train + sliding window      

  if training_method == 0:
    gamma = 1
    valid_split = 1
  else:
    gamma = hyperparams[5]  # damping coefficient
    valid_split = hyperparams[6]  # validation set split: 0.2
  print('gamma = ', gamma)
  print('validation split = ', valid_split)

  slidewindow_len = hyperparams[7]  # Nb of previous days considered: 60
  print('Sliding Window Size (days) = ', slidewindow_len)
  #----------------------

  #------- Reshape  ------- 
  #Usefull constants
  D, H, T = train_trips.shape  # (Day, hour-of-the-day, bus-stop-ID)
  train_trips = train_trips.reshape(-1,T)
  train_timeliness = train_timeliness.reshape(-1,T)
  N, T = train_trips.shape  # (Contiguous-trips, bus-stop-ID)

  #------- Split Dataset ------- 
  split_idx = int(train_trips.shape[0] * valid_split)
  x_tr, x_seq = train_trips[:-split_idx], train_trips[-split_idx:]
  y_tr, y_seq = train_timeliness[:-split_idx], train_timeliness[-split_idx:]


  print('==== Dataset ====')
  print(f'X: Timestamp of scheduled bus stop, Y: Timeliness on that day at that bus stop')
  print(x_seq.shape, y_seq.shape)
  print('split index', train_trips.shape[0] - split_idx)
  print(x_tr.shape, x_seq.shape, y_tr.shape, y_seq.shape)

  #-- Check clean Data --
  if x_seq.shape[0] == 0: 
    raise Exception('Empty validation set, increase "valid_split"')
  if slidewindow_len > x_seq.shape[0] - H: 
    slidewindow_len = x_seq.shape[0] - H
    print(f'Sliding Window reduced to: {slidewindow_len/H} days')
  #----------------------


  #Init
  state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
  accu1 = 0
  accu2 = 0

  if training_method == 1:  # Batch mode: use training set to establish priors
    # Train
    pi_batch, P_batch, Q_batch = parametrization_2d(state_dict, x_tr, y_tr, knots, stop_homogeneous, time_homogeneous)


  if training_method == 0 or training_method == 1:  # Sliding window: use sequential data from test set to modify the baseline
    for i in range(slidewindow_len, N):
      # Create a validation set   
      X_seq = x_seq[i-slidewindow_len:i,:]
      Y_seq = y_seq[i-slidewindow_len:i,:]
      X_tst = x_seq[i,:]
      Y_tst = y_seq[i,:]
      # Train
      pi, P, Q = parametrization_2d(state_dict, X_seq, Y_seq, knots, stop_homogeneous, time_homogeneous)
      if training_method == 1:
        pi = gamma * pi + (1-gamma) * pi_batch
        P = gamma * P + (1-gamma) * P_batch
        Q = gamma * Q + (1-gamma) * Q_batch
      

      #Predict the complete sequence of states for the following trip given a trip
      future_states = pred_stop(Y_seq[-1], state_dict, pi, P, Q, X_tst, knots, stop_homogeneous, time_homogeneous, alpha)
      for t in range(T):
        if Y_tst[t] == future_states[t]:
            accu1 += 1

      #Predict the state of the next stop given the previous trip and the state of the current stop
      future_states = pred_stop(Y_seq[-1], state_dict, pi, P, Q, X_tst, knots, stop_homogeneous, time_homogeneous, alpha, Y_tst)
      for t in range(T):
          if Y_tst[t] == future_states[t]:
              accu2 += 1

      if i%100==0: print(f'accu1: {accu1}, accu2: {accu2}', end='\r')


  accu1 = accu1 /((N - slidewindow_len) * T)
  accu2 = accu2 /((N - slidewindow_len) * T)
  print('==== Results ====')
  print('Next Sequence:', accu1)
  print('Next State', accu2)
  print()
  print()

In [62]:
# Set Hyperparemeters
homogeneous = 2           # 0:horizontal-only, 1:vertical-only, 2:2D homogeneous, 3:2D inhomogeneous
inhomogeneous_method = 0  # 0:Peak periods, 1:Hourly
direction = 0             # 0:Horizontal, 1:Vertical, 2:2D
alpha = 0.5               # [0,0.5]:vertical (time) axis dominates, [0.5,1]:horizontal (stop_id) axis dominates
training_method = 0       # 0:sliding window only, 1: 
gamma = 0.9               # [0,0.5]:static set of routes dominate, [0.5,1]:set of recent routes dominates
valid_split = 0.2         # (0,1]: subset of training set on which predictions are made (empty validation set is not allowed)
slidewindow_len = 60      # Nb of days from which the priors and transition matrix are computed

hyperparams = [homogeneous, inhomogeneous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
data = [X2_train, y2_train]

for homogeneous in range(4):
  hyperparams = [homogeneous, inhomogeneous_method, direction, alpha, training_method, gamma, valid_split, slidewindow_len]
  experiment(data, hyperparams)


last continuous-time timestamp: 25.433333333333334
==== Hyperparameters ====
Homogenous = 0
Stop Homogenous = True
Time Homogenous = False
Inhomogeneous Method = 0
Conditional Dependance Direction = 0
alpha = 1
Training Method = 0
gamma =  1
validation split =  1
Sliding Window Size (days) =  60
==== Dataset ====
X: Timestamp of scheduled bus stop, Y: Timeliness on that day at that bus stop
(7334, 9) (7334, 9)
split index 0
(0, 9) (7334, 9) (0, 9) (7334, 9)
==== Results ====
Next Sequence: 0.4763694131304799
Next State 0.7979561909999083


last continuous-time timestamp: 25.433333333333334
==== Hyperparameters ====
Homogenous = 1
Stop Homogenous = False
Time Homogenous = True
Inhomogeneous Method = 0
Conditional Dependance Direction = 0
alpha = 1
Training Method = 0
gamma =  1
validation split =  1
Sliding Window Size (days) =  60
==== Dataset ====
X: Timestamp of scheduled bus stop, Y: Timeliness on that day at that bus stop
(7334, 9) (7334, 9)
split index 0
(0, 9) (7334, 9) (0, 9) (7

##Old algo: resolve conflict by merging with above

In [ ]:
import math

def parametrization(state_dict, X_tr, y_tr, knots, time_homogeneous = True, stop_homogeneous = True):
      
    N, T = X_tr.shape

    if not time_homogeneous:
        transition_matrix = np.zeros((len(knots) - 1,3,3))
        pi = np.zeros((len(knots) - 1, 3))

    elif not stop_homogeneous:
        transition_matrix = np.zeros((T - 1,3,3))
        pi = np.zeros(3)
    
    else:
        transition_matrix = np.zeros((3,3))
        pi = np.zeros(3)

    for i in range(N):

        actual_state = state_dict[y_tr[i,0]] 

        if not time_homogeneous:
            for k in range(len(knots) - 1):
                if (X_tr[i,0] >= knots[k]) and (X_tr[i,0] < knots[k + 1]):
                    dtw = k
            pi[dtw, actual_state] += 1
  
        else:
            pi[actual_state] += 1

        for j in range(T - 1):
            actual_state = state_dict[y_tr[i,j]]
            next_state = state_dict[y_tr[i,j + 1]]

            if not time_homogeneous:
                for k in range(len(knots) - 1):
                    if X_tr[i,j] >= knots[k] and X_tr[i,j]< knots[k + 1]:
                        dtw = k
                transition_matrix[dtw, actual_state, next_state] +=  1
            
            elif not stop_homogeneous:
                transition_matrix[j, actual_state, next_state] +=  1

            else:
                transition_matrix[actual_state, next_state] +=  1

    #Normalization
    if not time_homogeneous:

        for k in range(0, len(knots) - 1):

            if sum(pi[k,:]) == 0:
                pi[k,:] = [1/3, 1/3, 1/3]
            else:
                pi[k,:] = pi[k,:] / sum(pi[k,:])
            
            if transition_matrix[k,:,:].sum(axis = 1)[0] == 0:
                transition_matrix[k,0,:] = [1/3, 1/3, 1/3]
            if transition_matrix[k,:,:].sum(axis = 1)[1] == 0:
                transition_matrix[k,1,:] = [1/3, 1/3, 1/3]
            if transition_matrix[k,:,:].sum(axis = 1)[2] == 0:
                transition_matrix[k,2,:] = [1/3, 1/3, 1/3]
                
            transition_matrix[k,:,:] = (transition_matrix[k,:,:].T/transition_matrix[k,:,:].sum(axis = 1)).T

    elif not stop_homogeneous:

        for j in range(T - 1):
            pi = pi/sum(pi)
            
            if transition_matrix[j,:,:].sum(axis = 1)[0] == 0:
                transition_matrix[j,0,:] = [1/3, 1/3, 1/3]
            if transition_matrix[j,:,:].sum(axis = 1)[1] == 0:
                transition_matrix[j,1,:] = [1/3, 1/3, 1/3]
            if transition_matrix[j,:,:].sum(axis = 1)[2] == 0:
                transition_matrix[j,2,:] = [1/3, 1/3, 1/3]
                
            transition_matrix[j,:,:] = (transition_matrix[j,:,:].T/transition_matrix[j,:,:].sum(axis = 1)).T

    else:      
        pi = pi/sum(pi)
        
        #Check first divison by 0
        if transition_matrix.sum(axis = 1)[0] == 0 or transition_matrix.sum(axis = 1)[1] == 0 or transition_matrix.sum(axis = 1)[2] == 0:
            print('error: one or more states not seen in training set. Increase slidewindow_len.')
        else:
            transition_matrix = (transition_matrix.T/transition_matrix.sum(axis = 1)).T

    return pi, transition_matrix

def pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, time_homogeneous = True, stop_homogeneous = True):

    future_states = []

    if not time_homogeneous:

        for k in range(len(knots) - 1):
            if X_tst[0] >= knots[k] and X_tst[0] < knots[k + 1]:
                dtw = k
        first_stop_state = np.random.choice(np.array(list(state_dict.keys())), p = pi[dtw, :])
    
    else:
        first_stop_state = np.random.choice(np.array(list(state_dict.keys())), p = pi)


    future_states.append(first_stop_state)

    for i in range(len(X_tst) - 1):
        current_state = state_dict[future_states[-1]]

        if not time_homogeneous:

            for k in range(len(knots) - 1):
                if X_tst[i] >= knots[k] and X_tst[i] < knots[k + 1]:
                    dtw = k
            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[dtw, current_state, :])
        
        elif not stop_homogeneous:
            
            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[i, current_state, :])

        else:

            next_state = np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[current_state, :])

        future_states.append(next_state)

    return future_states

def pred_next_stop_delay(current_state, state_dict, transition_matrix, x_tst, knots, t, time_homogeneous = True, stop_homogeneous = True):

      if not time_homogeneous:

          for k in range(len(knots) - 1):
              if x_tst >= knots[k] and x_tst < knots[k + 1]:
                  dtw = k
          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[dtw, state_dict[current_state], :])
      
      elif not stop_homogeneous:
          
          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[t, state_dict[current_state], :])

      else:

          return np.random.choice(np.array(list(state_dict.keys())), p = transition_matrix[state_dict[current_state], :])

In [ ]:
#----- Parameters ----- 
slidewindow_len = 250 #Number of previous days to train on
time_homogeneous = True
stop_homogeneous = False
if not time_homogeneous and not stop_homogeneous:
    print('time inhomogeneous and stop inhomogeneous markov chains not implemented, keep time_homogeneous or stop_homogeneous True')
    exit() 
knots = [0, 21600, 32400, 50400, 64800, 93600] #DTWs time limits (for inhomogeneous chains)
#----------------------

#Init
state_dict = {'Early': 0, 'On Time': 1, 'Late': 2}
accu1 = 0
accu2 = 0

#Usefull constants
N, T = X_train.shape

for k in range(slidewindow_len, N):

    X_tr = X_train[k-slidewindow_len:k,:]
    Y_tr = y_train[k-slidewindow_len:k,:]
    X_tst = X_train[k,:]
    y_tst = y_train[k,:]
    pi, transition_matrix = parametrization(state_dict, X_tr, Y_tr, knots, time_homogeneous, stop_homogeneous)


    #Predict the next trip sequence of states
    future_states = pred_next_trip_delays(state_dict, pi, transition_matrix, X_tst, knots, time_homogeneous, stop_homogeneous)
    
    for t in range(T):

        if y_tst[t] == future_states[t]:
            accu1 += 1
    
    #Predict the state of the next stop
    if not time_homogeneous:

        for k in range(len(knots) - 1):
            if X_tst[0] >= knots[k] and X_tst[0] < knots[k + 1]:
                dtw = k
        first_state = np.random.choice(np.array(list(state_dict.keys())), p = pi[dtw, :]) #Prediction of the first stop state is random
    
    else:
        first_state = np.random.choice(np.array(list(state_dict.keys())), p = pi) #Prediction of the first stop state is random

    if y_tst[0] == first_state:
        accu2 += 1

    for t in range(0, T - 1):
        next_state = pred_next_stop_delay(y_tst[t], state_dict, transition_matrix, X_tst[t], knots, t, time_homogeneous, stop_homogeneous)
        
        if y_tst[t] == next_state:
            accu2 += 1


accu1 = accu1 /((N - slidewindow_len) * T)
accu2 = accu2 /((N - slidewindow_len) * T)
print(accu1)
print(accu2)